Tech Project 2 - Ensemble Methods

Diogo Pessoa


In [17]:
import pandas as pd

In [18]:
combined_data=pd.read_csv('combined_data.csv').set_index('video_id')
label = combined_data['label']
features = combined_data.drop(['label'],axis=1)

## Scaling Dataset features

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features.values, label.values, test_size=0.2, random_state=0)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [12]:
print('check_data_split',[x_train.shape,x_test.shape,y_train.shape,y_test.shape])

check_data_split [(123653, 38), (30914, 38), (123653,), (30914,)]


In [ ]:
# code here
from sklearn.decomposition import PCA
pca = PCA(n_components=34)
x_train_Trans=pca.fit_transform(x_train)
x_test_Trans=pca.transform(x_test)

## Voting Regressor to compare different models

* [skelarn Voting Regressor](https://scikit-learn.org/stable/modules/ensemble.html#voting-regressor)
* [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
* [Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
* [Gradient Boosting Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)
* 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

n_estimators=140 
# Training classifiers
grad_boosting_regressor = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators)
ada_boosting_regressor = AdaBoostRegressor(random_state=42, n_estimators=n_estimators)
random_forest_regressor = RandomForestRegressor(random_state=42, n_estimators=n_estimators)
bagging_regressor = BaggingRegressor(random_state=1, n_estimators=n_estimators)
linear_regr = LinearRegression()
voting_reg = VotingRegressor(estimators=[('gb', grad_boosting_regressor), ('rf', random_forest_regressor), ('lr', linear_regr), ('ada_b', ada_boosting_regressor), ('bagging_r', bagging_regressor)]) 

In [ ]:
# Traing models - GradientBoostingRegressor, RandomForestRegressor, LinearRegression, AdaBoostRegressor, BaggingRegressor, VotingRegressor
grad_boosting_regressor.fit(x_train_Trans, y_train)
ada_boosting_regressor.fit(x_train_Trans, y_train)
bagging_regressor.fit(x_train_Trans, y_train)
random_forest_regressor.fit(x_train_Trans, y_train)
linear_regr.fit(x_train_Trans, y_train)
voting_reg.fit(x_train_Trans, y_train)

In [ ]:
gdb_prediction = grad_boosting_regressor.predict(x_test_Trans)
rf_prediction = random_forest_regressor.predict(x_test_Trans)
lg_prediction = linear_regr.predict(x_test_Trans)
ada_b_prediction = ada_boosting_regressor.predict(x_test_Trans)
bagging_r_prediction = bagging_regressor.predict(x_test_Trans)
voting_reg_prediction = voting_reg.predict(x_test_Trans)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

print('mean squared error', 'r2 score', 'explained variance score')
print('GradientBoostingRegressor',mean_squared_error(y_test, gdb_prediction),r2_score(y_test, gdb_prediction), explained_variance_score(y_test, gdb_prediction))
print('RandomForestRegressor',mean_squared_error(y_test, rf_prediction),r2_score(y_test, rf_prediction), explained_variance_score(y_test, rf_prediction))
print('LinearRegression',mean_squared_error(y_test, lg_prediction),r2_score(y_test, lg_prediction), explained_variance_score(y_test, lg_prediction))
print('AdaBoostRegressor',mean_squared_error(y_test, ada_b_prediction),r2_score(y_test, ada_b_prediction), explained_variance_score(y_test, ada_b_prediction))
print('BaggingRegressor',mean_squared_error(y_test, bagging_r_prediction),r2_score(y_test, bagging_r_prediction), explained_variance_score(y_test, bagging_r_prediction))
print('VotingRegressor',mean_squared_error(y_test, voting_reg_prediction),r2_score(y_test, voting_reg_prediction), explained_variance_score(y_test, voting_reg_prediction))

In [ ]:
from  matplotlib import pyplot as plt

plt.figure(figsize=(14, 10))
# plt.plot(rf_prediction, "b^", label="RandomForestRegressor")
# plt.plot(lg_prediction, "ys", label="LinearRegression")
# plt.plot(gdb_prediction, "gd", label="GradientBoostingRegressor")
plt.plot(voting_reg_prediction, "gd", ms=10, label="VotingRegressor")
plt.plot(ada_b_prediction, "r*", label="AdaBoostRegressor")
plt.plot(bagging_r_prediction, "b*", label="BaggingRegressor")

plt.tick_params(axis="x", which="both", bottom=False, top=False, labelbottom=False)
plt.ylabel("predicted")
plt.xlabel("training samples")
plt.legend(loc="best")
plt.title("Regressor predictions and their average")

plt.show()

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Only tune the max depth of the trees in the RF hyperparameter.
grid = GridSearchCV(estimator=AdaBoostRegressor(), param_grid={'n_estimators':[140],'random_state':[5,20,42]},cv=5)
grid.fit(x_train_Trans, y_train)
grid.best_params_

NameError: name 'AdaBoostRegressor' is not defined

In [ ]:
%pip install optuna

In [23]:
import optuna
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
# Define the objective function
def objective(trial):
    C = trial.suggest_loguniform('C', 0.001, 100)
    gamma = trial.suggest_uniform('gamma', 0.0001, 1)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])

    model = SVR(C=C, gamma=gamma, kernel=kernel)
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    mse = mean_squared_error(y_test, pred)
    return mse



In [ ]:
# Run the optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial.params)

[I 2024-02-16 14:18:23,400] A new study created in memory with name: no-name-b7ac68a9-7a31-4ae0-a4a4-8cf5cba68618
/var/folders/s_/6d26m9ld7d1bdywv1jdxvg500000gp/T/ipykernel_25948/4274395389.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 100)
/var/folders/s_/6d26m9ld7d1bdywv1jdxvg500000gp/T/ipykernel_25948/4274395389.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gamma = trial.suggest_uniform('gamma', 0.0001, 1)
